In [10]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pmlb as dsets
import numpy as np
import pickle as pkl
from os.path import join as oj
from copy import deepcopy
import pandas as pd
from numpy import array as arr

# sklearn models
sys.path.append('../scores')
import scores
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import interactions
from scipy import interpolate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
data_dir = '/scratch/users/vision/data/pmlb'
out_dir = '/scratch/users/vision/chandan/pmlb/'
dset_names = deepcopy(dsets.classification_dataset_names)
dset_names.remove('kddcup') # # remove biggest dset
dset_names.remove('mnist') # # remove biggest dset
dset_names.remove('poker') # # remove biggest dset

dset_names = np.array(dset_names)
random_state = 42 # for each train_test_split
classification_only = False

# train models
this code will save all the **classification** dsets and fit very quick models to each of them

In [ ]:
from train import fit_logit_and_rfs
fit_logit_and_rfs(dset_names, data_dir, out_dir, classification_only, random_state=random_state)

# analyze results

In [5]:
results = pd.DataFrame(pkl.load(open(oj(out_dir, 'classification_results_orig_seeded.pkl'), 'rb')))

# plot
# sns.boxplot(data=[results['logit_test_score'], results['rf_test_score']])
# plt.xticks([0, 1], ['Logistic Regression', 'Random Forest'])
# plt.ylabel('Test Accuracy')
# plt.show()

In [6]:
results.shape

(163, 4)

# load all scores results

In [ ]:
for dset_num in range(results.shape[0]):
    fit_altered(data_dir, out_dir, dset_num=dset_num, random_state=42)
    
